https://catalog.data.gov/dataset/3-year-recidivism-for-offenders-released-from-prison

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [62]:
#df  = pd.read_csv('/Users/matanefron/Desktop/df-sf-32/DS-SF-32/lessons/lesson-6/temp_data.txt', sep='\t')
df = pd.read_csv('/Users/matanefron/Downloads/3-Year_Recidivism_for_Offenders_Released_from_Prison.csv')

In [29]:
#data view
#df.head(20)
df[df['Offender']==1].head()

,Offender,Recidivism Reporting Year,Race - Ethnicity,Sex,Age At Release,Convicting Offense Classification,Convicting Offense Type,Convicting Offense Subtype,Release Type,Main Supervising District,Recidivism - Return to Prison,Recidivism Type,Days to Recidivism,New Conviction Offense Classification,New Conviction Offense Type,New Conviction Offense Sub Type,Part of Target Population
0,1,FY13,White - Non-Hispanic,M,Under 25,D Felony,Violent,Assault,Parole,4JD,Yes,Tech,16.0,NaN,NaN,NaN,Yes
1,1,FY13,White - Non-Hispanic,M,55 and Older,D Felony,Public Order,OWI,Parole,7JD,Yes,Tech,19.0,NaN,NaN,NaN,Yes
2,1,FY13,White - Non-Hispanic,M,25-34,D Felony,Property,Burglary,Parole,5JD,Yes,Tech,22.0,NaN,NaN,NaN,Yes
3,1,FY13,White - Non-Hispanic,M,55 and Older,C Felony,Drug,Trafficking,Parole,8JD,Yes,Tech,25.0,NaN,NaN,NaN,Yes
4,1,FY13,Black - Non-Hispanic,M,25-34,D Felony,Drug,Trafficking,Parole,3JD,Yes,Tech,26.0,NaN,NaN,NaN,Yes


In [51]:
print df.size
df.groupby(['Recidivism']).agg({'Offender':[np.size]})


307080


,Offender
,size
Recidivism,
False,11656
True,5404


In [65]:
#Set recidivism to true or false
df['Recidivism'] = df['Recidivism - Return to Prison']=='Yes'
#Change it to binary
df.Recidivism = df.Recidivism.astype(int)


In [38]:
df.dtypes

Offender                                   int64
Recidivism Reporting Year                 object
Race - Ethnicity                          object
Sex                                       object
Age At Release                            object
Convicting Offense Classification         object
Convicting Offense Type                   object
Convicting Offense Subtype                object
Release Type                              object
Main Supervising District                 object
Recidivism - Return to Prison             object
Recidivism Type                           object
Days to Recidivism                       float64
New Conviction Offense Classification     object
New Conviction Offense Type               object
New Conviction Offense Sub Type           object
Part of Target Population                 object
dtype: object

In [66]:
X = df[['Race - Ethnicity','Sex','Age At Release ','Convicting Offense Classification','Convicting Offense Type','Convicting Offense Subtype','Release Type']]

In [67]:
y = df['Recidivism'].values